# Sistema de Recomendación

### Descripción del problema

El problema que se va a tratar en este proyecto consiste en la realización de un sistema de recomendación de productos de una aplicación de un comercio. Se quiere aparezca una sección de productos recomendados que le podrían interesar a un determinado usuario.

### Investigación

Tras realizar una investigación con la implementación de sistemas de recomendación por usuarios se ha encontrado una variedad de formas de implementar el sistema. Las más comúnes son:

#### Filtrado Colaborativo

Este método se basa en la colaboración entre múltiples usuarios para hacer recomendaciones. Asume que si un usuario A tiene gustos similares a los de un usuario B, A probablemente disfrutará ítems que B ha disfrutado.

Tipos:
- Basado en Usuarios: Recomienda ítems basándose en la similitud entre usuarios.
- Basado en Ítems: Recomienda ítems basándose en la similitud entre los propios ítems.


#### Filtrado Basado en Contenido

Utiliza las características de los ítems y el perfil del usuario para hacer recomendaciones. Se basa en la descripción de los ítems y en un perfil del usuario que indica sus preferencias.

Ejemplo: Si a un usuario le gusta un libro de ciencia ficción, el sistema recomendará otros libros de ciencia ficción.

#### Sistemas Híbridos

Combina múltiples técnicas de recomendación para mejorar la precisión y superar las limitaciones de los sistemas individuales. Por ejemplo, puede combinar filtrado colaborativo y basado en contenido.

Ejemplo: Netflix utiliza un sistema híbrido que combina el filtrado colaborativo y basado en contenido para ofrecer recomendaciones más precisas.

#### Recomendadores Basados en Redes Neuronales

Utilizan redes neuronales profundas para aprender patrones complejos a partir de los datos de usuarios y ítems. Capturan relaciones no lineales y mejoran con grandes cantidades de datos.

Ejemplo: Modelos como el autoencoder variacional o redes neuronales convolucionales aplicadas a datos de recomendación. 

#### Opción elegida

La opción elegida para implementar son los **filtros colaborativos** ya que dada las necesidades del sistema y tratandose de un sistema de productos de un comercio se considera que es la que mejor resultados puede proporcionar

### Código

- En la variable userInput es necesario guardar la información de los items que ha puntuado el usuario logueado en el sistema.
- En las variables items_df debe estar la información de los items que hay en el sistema y en ratings_df la información de las puntuaciones que otros usuarios (que no sea el logueado) han dado a los distintos items de la aplicación.
- Posteriormente se calcula el coeficiente de Pearson entre los distintos usuarios del sistema, es un coeficiente entre -1 y 1 que arroja el grado de similitud entre usuarios.
- Una vez se ha calculado el coeficiente de Pearson a los items de los usuarios con más parecido se les calcula y peso y se muestran los items que más probabilidad tienen de gustar al usuario logueado.

Este sistema de recomendación se puede emplear como base de distintos sistemas, es decir, por ejemplo si se quisiera tener un sistema de recomendación por países, únicamente sería necesario modificar la información que devuelven los servicios y el sistema funcionaría correctamente.

#### Importamos las librerías necesarias

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt

#### Hacemos los GET a los servicios

Para obtener los datos necesarios para el filtro colaborativo, se va a emplear la librería requests para hace un GET a un servicio, después se parseará el JSON que devuelve el servicio y por último con la librería de pandas se generarán los DataFrames necesarios.

Para este ejemplo no se van a realizar GET realmente y se van a simular cargando los datos desde ficheros

In [31]:
# Generamos un dataframe con los items de propio usuario
userInput = [
    {'item':'6B475DF6-5D69-4756-94AD-FBF3ADF921AB', 'rating':5.0},
    {'item':'0443E87F-995B-4196-82AA-9CB98D7B1509', 'rating':4.0},
    {'item':'A4F410BB-717C-4F57-B2FF-E249C95EB60F', 'rating':3.0},
    {'item':'5D1D782B-9CF3-4670-B8F3-A489A5619270', 'rating':2.0},
    {'item':'70B542A7-A72B-47EC-A1D3-735C2D32B8CF', 'rating':2.0},
    {'item':'30E80572-F83A-4C8E-9902-F0D46966B5FA', 'rating':1.0},
    {'item':'D0E70EDF-AB9D-4863-AB91-D1B006C61FC8', 'rating':3.0},
    {'item':'36F2958A-E6B5-432C-A56C-A43F8EE63DF4', 'rating':5.0},
    {'item':'0D979A9C-F2DD-428B-85C8-A90034E81764', 'rating':2.0},
    {'item':'C3DC290F-E643-4CCB-B40F-D5DBA5D6F014', 'rating':2.0},
    {'item':'1939E2D9-274F-4A2C-A508-0ED732ABCE74', 'rating':3.0},
    {'item':'92AE8EE9-60DB-4923-8D8D-73F383561AA1', 'rating':2.5},
    {'item':'05D01739-67E9-42F0-9E7B-ADE0D6D2AA2C', 'rating':4.3},
    {'item':'37F0555A-177E-464D-9565-3FB972891048', 'rating':5.0},
    {'item':'352628B3-1A5D-48D5-B865-9264BB911BBC', 'rating':1.5},
    {'item':'2F05212F-8941-4AA5-AECF-9060C72F0413', 'rating':1.0},
    {'item':'7DF25965-12B7-4B18-A194-4F090D9281E2', 'rating':0.0},
    {'item':'3A422B1F-CD61-4A0B-A536-49CDBB4198B0', 'rating':4.5},
    {'item':'F9ACAD1B-C0AF-4B09-9211-D79D0A3E4612', 'rating':4.5},
    {'item':'C0B9D63C-D24B-4F34-B7A6-815453341300', 'rating':3.5},
    {'item':'1EB72CB7-6F26-4011-B6AD-8096DAA5EF2C', 'rating':3.0},
    {'item':'E7B23C58-E1ED-4F73-8C85-65AA717D0CD7', 'rating':5.0},
    {'item':'F1A5B63D-AC9F-4B62-B5AC-40C7E853AE0C', 'rating':5.0},
    {'item':'69296A1F-54E0-490D-AEEC-3B5194DE2D46', 'rating':2.5},
    {'item':'D6CB6F6F-3F19-4B72-819A-3D2D44DE9421', 'rating':3.5},
    {'item':'8293CAEB-D806-49B9-92CC-209F521DA119', 'rating':1.5}
] 
inputItems = pd.DataFrame(userInput)

In [76]:
# Realizamos los servicios y generamos un dataframe con los items y sus ids
# Y otro dataframe con los usuarios, los items y las puntuaciones dadas
json_products = open('./data/productos.json')
products = json.load(json_products)
products_df = pd.json_normalize(products['productos'])
products_df.head()

,EntityCode,Nombre
0,A6C395F7-DE27-482C-A814-119AA4C1E5CD,El consumidor digital
1,19BE7C57-2024-4094-BE6C-BE98B61DCCE0,Uso de redes sociales
2,6EA97840-4CC0-4B1D-A78C-50DC9360E69D,Streaming
3,E1E397E8-9D5E-4336-9F2F-77BD6150939C,Estudio de Mercado del Ecommerce
4,5F9C1578-06A4-4DC8-8FFE-16F703AB8B09,Estudio de Mercado del hábitos por internet


In [75]:
# Obtenemos las puntuaciones dadas por los usuarios de todos los productos que tenemos en el comercio
json_rating = open('./data/pedidos.json')
rating = json.load(json_rating)
ratings_df = pd.json_normalize(rating['pedidos'])
ratings_df.head()

,IdCliente,IdItem,Rating
0,6CC3C382-DB89-4807-97CA-B0F99F4A2B29,0A80A6CF-F961-4EF7-9A90-2F8C7582A87B,5
1,417E9FAD-50AC-4A43-BC31-88E52D779DAF,9451657A-6166-42DF-BAAE-E03A2C9F609B,5
2,10E53D0C-B123-475F-83EE-C5EA1B0A15C6,2FC651C9-C7FF-4591-95CF-81683562FD91,5
3,7043883F-C06E-4F9C-AE89-25B1ED565636,C2482313-7A27-4A53-8AB8-43DFE183E997,2
4,8B871F41-D7B0-48DE-B14B-033DFB71A404,AF750425-6C23-4285-9617-9FBD33E731A8,3


A continuación, se combina el dataframe de todos los productos con los productos que el usuario ha dado una puntuación y se muestran para tenerlos identificados

In [74]:
inputId = products_df[products_df['EntityCode'].isin(inputItems['item'].tolist())]
inputItems = pd.merge(inputId, inputItems, how='inner', left_on='EntityCode', right_on='item')
inputItems.head()

,EntityCode_x,Nombre_x,EntityCode_y,Nombre_y,item,rating
0,92AE8EE9-60DB-4923-8D8D-73F383561AA1,Fororo SIN MARCA PRESENTACION NO REPORTADA,92AE8EE9-60DB-4923-8D8D-73F383561AA1,Fororo SIN MARCA PRESENTACION NO REPORTADA,92AE8EE9-60DB-4923-8D8D-73F383561AA1,2.5
1,05D01739-67E9-42F0-9E7B-ADE0D6D2AA2C,Fororo OTRA Bolsa 900 grs,05D01739-67E9-42F0-9E7B-ADE0D6D2AA2C,Fororo OTRA Bolsa 900 grs,05D01739-67E9-42F0-9E7B-ADE0D6D2AA2C,4.3
2,37F0555A-177E-464D-9565-3FB972891048,Fororo OTRA SIN PRESENTACION,37F0555A-177E-464D-9565-3FB972891048,Fororo OTRA SIN PRESENTACION,37F0555A-177E-464D-9565-3FB972891048,5.0
3,36F2958A-E6B5-432C-A56C-A43F8EE63DF4,Refresco Colas Negras BIG COLA OTRA PRESENTACI...,36F2958A-E6B5-432C-A56C-A43F8EE63DF4,Refresco Colas Negras BIG COLA OTRA PRESENTACI...,36F2958A-E6B5-432C-A56C-A43F8EE63DF4,5.0
4,1EB72CB7-6F26-4011-B6AD-8096DAA5EF2C,REFRESCOS DE SABORES FRESCOLITA 350 ml Botella...,1EB72CB7-6F26-4011-B6AD-8096DAA5EF2C,REFRESCOS DE SABORES FRESCOLITA 350 ml Botella...,1EB72CB7-6F26-4011-B6AD-8096DAA5EF2C,3.0


Obtenemos las puntuaciones que otros usuarios han dado a los mismos objetos que el usuario de entrada

In [73]:
userSubset = ratings_df[ratings_df['IdItem'].isin(inputItems['item'].tolist())]
userSubset.head()

,IdCliente,IdItem,Rating
118,467390C6-A659-486F-8490-5A3179729A8B,C3DC290F-E643-4CCB-B40F-D5DBA5D6F014,3
154,67264CF6-D4B5-4D8E-80F3-C948D46A059C,1939E2D9-274F-4A2C-A508-0ED732ABCE74,3
345,DF26BF82-AD3B-45C9-9E4E-796D4751F92C,6B475DF6-5D69-4756-94AD-FBF3ADF921AB,4
364,467390C6-A659-486F-8490-5A3179729A8B,1939E2D9-274F-4A2C-A508-0ED732ABCE74,3
670,EE90A912-C20E-4663-933D-00F590AFDE57,6B475DF6-5D69-4756-94AD-FBF3ADF921AB,2


In [79]:
# Agrupamos los productos puntuados por cada uno de los clientes
userSubsetGroup = userSubset.groupby(['IdCliente'])
userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)

#### Algoritmo

Este algoritmo está basado en calcular la correlación de Pearson. Esta correlación Se utiliza para medir la fuerza de una asociación lineal entre dos variables y poder identificar los usuarios que tienen gustos similares entre sí. Los valores obtenidos por este algoritmo varían entre -1 y 1, siendo 1 la correlación positiva perfecta entre las entidades y -1 forma una correlación negativa perfecta.

In [80]:
pearsonCorrelationDict = {}
for name, group in userSubsetGroup:
    group = group.sort_values(by='IdItem')
    inputItems = inputItems.sort_values(by='item')
    nRatings = len(group)
    temp_df = inputItems[inputItems['item'].isin(group['IdItem'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['Rating'].tolist()
    #Calculemos la Correlación Pearson entre dos usuarios, x e y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    #Si el denominador es diferente a cero, entonces dividir, sino, la correlación es 0.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name[0]] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name[0]] = 0

Obtenemos un diccionario con las correlaciones obtenidas hacia cada usuario

In [81]:
pearsonCorrelationDict.items()

dict_items([('467390C6-A659-486F-8490-5A3179729A8B', -0.8660254037844387), ('DF26BF82-AD3B-45C9-9E4E-796D4751F92C', 0.7745966692414834), ('F22E8503-7449-44E3-8B96-DCACE7652C3E', 0.6546536707079771), ('CBA6A484-1DBE-4307-8E50-2BCA62490889', 1.0), ('EE90A912-C20E-4663-933D-00F590AFDE57', -1.0), ('05F83343-3DBE-4BDE-9650-3384FCF05751', 0), ('0902A7C1-D13A-4481-BBE6-10B23A430AC3', 0), ('185A7033-F3AD-477A-A10C-20C77CB53EFE', 0), ('230EAA8B-7C75-4A9E-9814-2B2DA3072EDD', 0), ('4769801D-014C-45F7-9BB1-A223E00BFFD8', 0), ('4F79D2FB-CD62-4407-B4F0-3CF5F6EAB7F1', 0), ('67264CF6-D4B5-4D8E-80F3-C948D46A059C', 0), ('70FE3215-50BA-4273-B40E-F07488D6A625', 0), ('82E5089B-EF63-4195-9174-7BFD38CFE69F', 0), ('9E8BF402-549F-4B2E-BEAC-CDB047A43CCB', 0), ('A03E817D-B57B-4965-B5EB-77099CC662BB', 0), ('ACF95AEF-669F-4ED0-9A38-22BF3D8A03A8', 0), ('B1C84743-B903-4FFB-B3B2-1EEBFD5B3026', 0), ('DC21A602-F777-433B-9CDA-2D72C9D0902F', 0), ('E071C0FF-36EE-4899-AF05-4D39CE9B5F5E', 0), ('E21BAB8A-288E-4753-AEC6-8AC71

#### Resultados del algoritmo

Generamos distintos dataframes con los valores obtenidos del algoritmos

In [82]:
#Generamos un dataframe con las correlaciones de Pearson calculadas
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,-0.866025,467390C6-A659-486F-8490-5A3179729A8B
1,0.774597,DF26BF82-AD3B-45C9-9E4E-796D4751F92C
2,0.654654,F22E8503-7449-44E3-8B96-DCACE7652C3E
3,1.000000,CBA6A484-1DBE-4307-8E50-2BCA62490889
4,-1.000000,EE90A912-C20E-4663-933D-00F590AFDE57


In [83]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
3,1.000000,CBA6A484-1DBE-4307-8E50-2BCA62490889
1,0.774597,DF26BF82-AD3B-45C9-9E4E-796D4751F92C
2,0.654654,F22E8503-7449-44E3-8B96-DCACE7652C3E
12,0.000000,70FE3215-50BA-4273-B40E-F07488D6A625
14,0.000000,9E8BF402-549F-4B2E-BEAC-CDB047A43CCB


In [84]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='IdCliente', how='inner')
topUsersRating.head()

,similarityIndex,userId,IdCliente,IdItem,Rating
0,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,46681402-B3EA-4E36-8DB2-9700E6EA6D7B,3
1,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,060BD7AF-7734-4E89-B314-5354B86CDD78,5
2,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,CAC40830-6EAC-4CBB-A4D4-60DA2C3126ED,3
3,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,7284F571-9E6F-4F52-9006-A95F9DF5035E,5
4,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,004C060F-EDBC-4580-8DA5-541C71C80089,5


In [85]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Rating']
topUsersRating['weightedRating'] = topUsersRating['weightedRating'].astype(float)
topUsersRating.head()

,similarityIndex,userId,IdCliente,IdItem,Rating,weightedRating
0,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,46681402-B3EA-4E36-8DB2-9700E6EA6D7B,3,3.0
1,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,060BD7AF-7734-4E89-B314-5354B86CDD78,5,5.0
2,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,CAC40830-6EAC-4CBB-A4D4-60DA2C3126ED,3,3.0
3,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,7284F571-9E6F-4F52-9006-A95F9DF5035E,5,5.0
4,1.0,CBA6A484-1DBE-4307-8E50-2BCA62490889,CBA6A484-1DBE-4307-8E50-2BCA62490889,004C060F-EDBC-4580-8DA5-541C71C80089,5,5.0


In [86]:
tempTopUsersRating = topUsersRating.groupby('IdItem').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
IdItem,,
004C060F-EDBC-4580-8DA5-541C71C80089,1.0,5.0
00EE6E6D-39EF-4758-BFFF-5A8973740A72,0.0,0.0
015EABBE-0165-496F-9DCB-1FFB1FA5FDFE,0.0,0.0
017E8DE6-706E-482C-85A4-C1DED96C69A3,0.0,0.0
02CD16A8-CE6F-4922-ABD6-40C163EEDF2A,0.0,0.0


In [87]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['item_id'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,item_id
IdItem,,
004C060F-EDBC-4580-8DA5-541C71C80089,5.0,004C060F-EDBC-4580-8DA5-541C71C80089
00EE6E6D-39EF-4758-BFFF-5A8973740A72,NaN,00EE6E6D-39EF-4758-BFFF-5A8973740A72
015EABBE-0165-496F-9DCB-1FFB1FA5FDFE,NaN,015EABBE-0165-496F-9DCB-1FFB1FA5FDFE
017E8DE6-706E-482C-85A4-C1DED96C69A3,NaN,017E8DE6-706E-482C-85A4-C1DED96C69A3
02CD16A8-CE6F-4922-ABD6-40C163EEDF2A,NaN,02CD16A8-CE6F-4922-ABD6-40C163EEDF2A


Ordenamos de mayor a menor los items con mayor puntuación calculada por el sistema de recomendación

In [88]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(5)

,weighted average recommendation score,item_id
IdItem,,
30E80572-F83A-4C8E-9902-F0D46966B5FA,10.686932,30E80572-F83A-4C8E-9902-F0D46966B5FA
004C060F-EDBC-4580-8DA5-541C71C80089,5.000000,004C060F-EDBC-4580-8DA5-541C71C80089
42A86388-3036-4656-8A61-F09EE422F166,5.000000,42A86388-3036-4656-8A61-F09EE422F166
E4CCB273-A9CA-4711-BC0A-ABB311F29305,5.000000,E4CCB273-A9CA-4711-BC0A-ABB311F29305
E163F3A2-4172-450E-B03A-AAFF2BE38B02,5.000000,E163F3A2-4172-450E-B03A-AAFF2BE38B02


Mostramos los 10 productos que el sistema de recomendación ha obtenido

In [90]:
items_df.loc[items_df['EntityCode'].isin(recommendation_df.head(10)['item_id'].tolist())]

,EntityCode,Nombre
849,BBD23D9A-7E41-4B73-BD2E-35459DC4E96C,Aceites EL REY PRESENTACION NO REPORTADA
2274,C01DABAA-30FC-4859-A0ED-8513E34D9B64,Chucherías Dulces PERUGINA SIN PRESENTACION
3140,BDFA4426-2ACC-4906-B512-B38B2360C13E,Embutidos (mortadela) CARACAS SIN PRESENTACION
3805,E163F3A2-4172-450E-B03A-AAFF2BE38B02,Harina Precocida de Maíz JUANA SIN PRESENTACION
5466,30E80572-F83A-4C8E-9902-F0D46966B5FA,Leche Liquida Refrigerada TACHIRA (PAISA) 1 L
5469,E4CCB273-A9CA-4711-BC0A-ABB311F29305,Leche Liquida Refrigerada TACHIRA (PAISA) OTRA...
6017,42A86388-3036-4656-8A61-F09EE422F166,Arroz OTRA 500 GRS
6807,B9C43AE9-3B52-4B4E-83C7-A6AA96EFCCA1,Mayonesa MARCA NO REPORTADA BOLSA 100 - 400 cc
7534,004C060F-EDBC-4580-8DA5-541C71C80089,Atún Enlatado NEPTUNO SIN PRESENTACION
10064,DC579D20-4DEE-49A6-A1E0-DE8E45C68095,Sopas de Lata OTRA 113 GRS


Los 10 productos recomendados son:
- Aceites EL REY PRESENTACION NO REPORTADA
- Chucherías Dulces PERUGINA SIN PRESENTACION
- Embutidos (mortadela) CARACAS SIN PRESENTACION
- Harina Precocida de Maíz JUANA SIN PRESENTACION
- Leche Liquida Refrigerada TACHIRA (PAISA) 1 L
- Leche Liquida Refrigerada TACHIRA (PAISA) OTRA
- Arroz OTRA 500 GRS
- Mayonesa MARCA NO REPORTADA BOLSA 100 - 400 cc
- Atún Enlatado NEPTUNO SIN PRESENTACION
- Sopas de Lata OTRA 113 GRS